In [ ]:
# Classification Model Notebook
# File: 04_classification_model.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.classification_model import ClassificationModel

# Load classification data
df = pd.read_csv('../data/processed/data_for_classification.csv')
print(f"Data shape: {df.shape}")

# Check class distribution
print("\nEngagement Rate Class Distribution:")
print(df['engagement_rate_class'].value_counts())
print("\nPercentage Distribution:")
print(df['engagement_rate_class'].value_counts(normalize=True) * 100)

# Prepare features
X = df[['followers', 'influence_score', 'country_encoded']]
y = df['engagement_rate_class']

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Initialize and train model
cls_model = ClassificationModel(random_state=42, test_size=0.2)

# Prepare data
X_train_scaled, X_test_scaled, y_train, y_test = cls_model.prepare_data(X, y)

# Train model
model = cls_model.train_model(X_train_scaled, y_train, n_estimators=100)

# Evaluate
results = cls_model.evaluate(X_test_scaled, y_test)

# Confusion Matrix
cls_model.plot_confusion_matrix(y_test, results['y_pred'])

# Class Distribution
cls_model.plot_class_distribution(y_test, results['y_pred'])

# Feature importance for classification
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
plt.title('Feature Importance for Classification Model')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

# Cross-tabulation of predictions
prediction_analysis = pd.DataFrame({
    'Actual': y_test,
    'Predicted': results['y_pred']
})

print("\nPrediction Analysis:")
print(pd.crosstab(prediction_analysis['Actual'], 
                  prediction_analysis['Predicted'],
                  rownames=['Actual'], 
                  colnames=['Predicted']))

# Save model
cls_model.save_model('../models/random_forest_classifier.pkl')

print("\nClassification model training completed and saved!")